In [1]:
#импорт
import pandas as pd
import requests
import json
import time
import math
import vk
import collections
import os

import vkauth
#vk_auth.py
#auth = {"id_app":'' , "login":'', "password":''}

session = vk.AuthSession(vkauth.auth['id_app'], vkauth.auth['login'], vkauth.auth['password'])
vk_api = vk.API(session)

pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
#logger
#TODO дополнительно писать лог в файл
#Вывести в отдельный модуль
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
#определяем название выходного файла для заведений
places_out_file = 'places_fn.csv'

#определяем название выходного файла для чекинов
checkins_out_file = 'checkins_fn.csv'

#определяем название выходного файла для пользователей
users_out_file = 'users_fn.csv'

#определяем название выходного файла для социального графа
friends_out_file = 'friends_fn.csv'

#определяем пространственные границы сбора данных
bounds = {'lower':{'lat': 59.937557, 'lng': 30.387023}, 'upper':{'lat': 59.942765, 'lng': 30.397724}}

#определяем временные границы сбора чекинов

In [4]:
#Папки
raw_data = '../../Data/RawData/'
processing_data = '../../Data/ProcessingData/'
result_data = '../../Data/CleanData/'

In [5]:
#Функции сбора заведений

deq = collections.deque(maxlen=4)
def trottling_request():
    deq.appendleft(time.time())
    if len(deq) == 4:
        # 3 запроса в секунду, если нужно - подождем
        time.sleep(max(1+deq[3]-deq[0], 0.5))
        

#задать сетку
def get_circle_centers(lower, upper, radius):
    """
    cover the search area with circles for radar search
    http://stackoverflow.com/questions/7477003/calculating-new-longtitude-latitude-from-old-n-meters
    :param lower: lower bound of area (westmost + southmost)
    :param upper: upper bound of area (eastmost + northmost)
    :param radius: specified radius, adapt for high density areas
    :return: list of circle centers that cover the area between lower/upper
    """
    r, coords = 6378, list()
    while lower[1] < upper[1]:
        tmp = lower[0]

        while tmp < upper[0]:
            coords.append([tmp, lower[1]])
            tmp += (0.25 / r) * (radius / math.pi)
        lower[1] += (0.25 / r) * (radius / math.pi) / math.cos(lower[00] * math.pi / radius)

    return coords

def getPlaces(lat, lon, radius=1, count=1000, offset=0):
    if count > 1000:
        raise Exception('Bad params: max of count = 1000')
        
    response = vk_api.places.search(latitude = lat, longitude = lon, radius = radius, count = count, offset = offset, version= 5.85)
    
    return response

#TODO писать в файл каждые n итераций
def allPlacesOffset(func, func_lat, func_lon):
    places = []
    place_count = -1
    offset = 0
    while place_count != len(places):
        trottling_request()
        
        try:
            response = func(func_lat, func_lon, radius = 1, count = 1000, offset = offset)
        except Exception:
            print("error")
            time.sleep(1)
            continue
          
        if place_count != response[0]:
            place_count = response[0]
        
        items = response[1:]
        offset = offset + len(items)
        
        places = places + items
    
    return places

In [6]:
circles = get_circle_centers([bounds["lower"]["lat"], bounds["lower"]["lng"]],  # southwest
                                       [bounds["upper"]["lat"], bounds["upper"]["lng"]],  # northeast
                                       150)

In [7]:
all_places = []
for lat, lng in log_progress(circles,1):
    places = allPlacesOffset(getPlaces, lat, lng)
    #Писат сразу в файлик
    all_places = all_places + places

VBox(children=(HTML(value=''), IntProgress(value=0, max=6)))

In [11]:
#save
places_df = pd.DataFrame(all_places)
places_df.to_csv(raw_data + places_out_file)

In [12]:
#Запускаем сбор чекинов

def getChekinsFromCoords(lat, lng, radius, count = 100, offset = 0):
    if count > 100:
        raise Exception('Bad params: max of count = 100')
    #response = requests.get('{host}/places.getCheckins?latitude={lat}&longitude={lon}&count={count}&offset={offset}&need_places=1'
    #                        .format(host=host,lat=lat, lon=lon, count=count, offset=offset))
    response = vk_api.places.getCheckins(latitude = lat, longitude = lng, radius = radius, count = count, offset = offset, version = 5.85)
    
    return response   

def getCheckinsFromPlace(place_id, count=100, offset = 0):
    #host = 'http://api.vk.com/method'
    if count > 100:
        raise Exception('Bad params: max of count = 100')
    #response = requests.get('{host}/places.getCheckins?latitude={lat}&longitude={lon}&count={count}&offset={offset}&need_places=1'
    #                        .format(host=host,lat=lat, lon=lon, count=count, offset=offset))
    response = vk_api.places.getCheckins(place = place_id, count = count, offset = offset)
    
    return response

def allChecinsCountOffsetFromCoords(func, lat, lng):
    checkins = []
    count_checins = -1
    offset = 0
    while count_checins != len(checkins): # posible endless loop for real vk api
        trottling_request()
        try:
            response = func(lat, lng, radius=1,count=100, offset=offset)
        except Exception as e:
            print(e)
            time.sleep(1)
            continue
        
        if count_checins != response[0]:
            count_checins = response[0]
        
        items = response[1:]
        
        if len(items) == 0:
            return checkins
        
        offset = offset + len(items)
        
       
        #if dict_checkins | set(new_members_id) == set_members_id != set(): # without new members
         #   print('WARNING: break loop', count_members, len(set_members_id))
          #  break
        
        checkins = checkins + items

    return checkins

def allChecinsCountOffsetFromPlace(func, place_id):
    checkins = []
    count_checins = -1
    offset = 0
    while count_checins != len(checkins): # posible endless loop for real vk api
        trottling_request()
        try:
            response = func(place_id,count=100, offset=offset)
        except Exception as e:
            print(e)
            time.sleep(1)
            continue
        
        if count_checins != response[0]:
            count_checins = response[0]
        
        items = response[1:]
        
        if len(items) == 0:
            return checkins
        
        offset = offset + len(items)
        
       
        #if dict_checkins | set(new_members_id) == set_members_id != set(): # without new members
         #   print('WARNING: break loop', count_members, len(set_members_id))
          #  break
        
        checkins = checkins + items

    return checkins

In [13]:
all_checkins = []
for lat, lng in log_progress(circles,1):
    checkins = allChecinsCountOffsetFromCoords(getChekinsFromCoords, lat, lng)
    all_checkins = all_checkins + checkins

VBox(children=(HTML(value=''), IntProgress(value=0, max=6)))

In [15]:
#save chekins
checkins_df = pd.DataFrame(all_checkins)
checkins_df.to_csv(raw_data + checkins_out_file)

In [ ]:
#скачать информацию по пользвателям

